In [1]:
import cv2
import pandas as pd
from typing import List

def create_annotated_video(
    df: pd.DataFrame, 
    labels_to_show: List[float],
    input_video_path: str,
    output_video_path: str,
    confidence_threshold: float = 0.3
):
    """
    Creates an annotated video showing bounding boxes and labels for the specified `labels_to_show`.
    In the last frame, the detection closest to the frame center is highlighted with a darker green bounding box
    and the label "Reptilian 1.0". After processing, the last annotated frame is displayed.
    
    Args:
        df (pd.DataFrame): DataFrame with columns:
            - image_idx: index of the frame
            - label: class label
            - confidence: detection confidence
            - x, y: bounding box center (in pixel coordinates or already converted)
            - w, h: bounding box width and height (in the same coordinate system)
        labels_to_show (List[float]): List of labels (classes) to display in the video.
        input_video_path (str): Path to the input video file.
        output_video_path (str): Path to the output (annotated) video file.
        confidence_threshold (float): Only draw bounding boxes with confidence above this value.
    """
    
    # Open video capture
    cap = cv2.VideoCapture(input_video_path)
    if not cap.isOpened():
        print(f"Could not open video file: {input_video_path}")
        return

    # Video properties
    width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps    = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Video writer
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))
    
    # Pre-filter the DataFrame once and group by frame index.
    # This avoids filtering the entire DataFrame for each frame.
    df_filtered = df[(df['confidence'] > confidence_threshold) & (df['label'].isin(labels_to_show))]
    grouped = df_filtered.groupby("image_idx")
    
    frame_idx = 0  
    last_frame = None  # To store the final annotated frame
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Retrieve detections for the current frame if available
        if frame_idx in grouped.groups:
            frame_df = grouped.get_group(frame_idx)
        else:
            frame_df = pd.DataFrame(columns=df.columns)
        
        # Process each detection in this frame
        for _, row in frame_df.iterrows():
            label = row['label']
            conf  = row['confidence']
            
            # Assuming the coordinates are in pixels.
            # If they are normalized, multiply by width/height as needed.
            x_center = row['x']
            y_center = row['y']
            w_box    = row['w']
            h_box    = row['h']
            
            x1 = int(x_center - w_box/2)
            y1 = int(y_center - h_box/2)
            x2 = int(x_center + w_box/2)
            y2 = int(y_center + h_box/2)
            
            if label == 2:
                background_color = (147, 20, 255)  # Pink for "Female"
                t = "Female"
            else:
                background_color = (255, 0, 0)       # Blue for "Male"
                t = "Male"
            
            cv2.rectangle(frame, (x1, y1), (x2, y2), background_color, 4)
            
            # Prepare text with background
            text = f"{t} {conf:.2f}"
            font = cv2.FONT_HERSHEY_SIMPLEX
            font_scale = 0.5
            thickness = 1
            y_label = max(0, y1 - 5)
            (text_width, text_height), baseline = cv2.getTextSize(text, font, font_scale, thickness)
            
            rect_x1 = max(x1, 0)
            rect_y1 = max(y_label - text_height - baseline, 0)
            rect_x2 = min(x1 + text_width, frame.shape[1])
            rect_y2 = min(y_label, frame.shape[0])
            
            cv2.rectangle(frame, (rect_x1, rect_y1), (rect_x2, rect_y2), background_color, cv2.FILLED)
            cv2.putText(frame, text, (rect_x1, rect_y2 - baseline), font, font_scale, (255, 255, 255), thickness, cv2.LINE_AA)
        
        # If this is the last frame, add the special "Reptilian" annotation.
        if frame_idx == total_frames - 1 and not frame_df.empty:
            # Determine frame center
            frame_center_x = frame.shape[1] // 2
            frame_center_y = frame.shape[0] // 2
            min_dist = float("inf")
            closest_row = None
            
            for _, row in frame_df.iterrows():
                # If coordinates are normalized, adjust accordingly.
                det_center_x = int(row['x'])
                det_center_y = int(row['y'])
                dist = ((det_center_x - frame_center_x) ** 2 + (det_center_y - frame_center_y) ** 2) ** 0.5
                if dist < min_dist:
                    min_dist = dist
                    closest_row = row
            
            if closest_row is not None:
                x_center = closest_row['x']
                y_center = closest_row['y']
                w_box    = closest_row['w']
                h_box    = closest_row['h']
                x1 = int(x_center - w_box/2)
                y1 = int(y_center - h_box/2)
                x2 = int(x_center + w_box/2)
                y2 = int(y_center + h_box/2)
                
                # Use a darker green (0, 128, 0)
                green_color = (0, 128, 0)
                cv2.rectangle(frame, (x1, y1), (x2, y2), green_color, 4)
                
                # Draw the "Reptilian 1.0" label
                text = "Reptilian 1.0"
                font = cv2.FONT_HERSHEY_SIMPLEX
                font_scale = 0.5
                thickness = 1
                y_label = max(0, y1 - 5)
                (text_width, text_height), baseline = cv2.getTextSize(text, font, font_scale, thickness)
                
                rect_x1 = max(x1, 0)
                rect_y1 = max(y_label - text_height - baseline, 0)
                rect_x2 = min(x1 + text_width, frame.shape[1])
                rect_y2 = min(y_label, frame.shape[0])
                
                cv2.rectangle(frame, (rect_x1, rect_y1), (rect_x2, rect_y2), green_color, cv2.FILLED)
                cv2.putText(frame, text, (rect_x1, rect_y2 - baseline), font, font_scale, (255, 255, 255), thickness, cv2.LINE_AA)
        
        out.write(frame)
        last_frame = frame.copy()
        frame_idx += 1
    
    cap.release()
    out.release()
    print(f"Annotated video saved as: {output_video_path}")
    
    # Display the last annotated frame
    if last_frame is not None:
        cv2.imshow("Last Annotated Frame", last_frame)
        cv2.waitKey(0)
        cv2.destroyAllWindows()


In [ ]:
import pandas as pd

# First load the CSV file into a DataFrame
df = pd.read_csv("/home/bohbot/Evyatar/exp/Joni_showoff/Rept/predict/results.csv")

# Then call your function, passing the actual DataFrame (df)
create_annotated_video(
    df,                       # <-- pass the DataFrame, not the CSV path
    labels_to_show=[0],                        # class_label
    input_video_path = "/home/bohbot/Evyatar/exp/Joni_showoff/Rertiles/4927592-hd_1920_1080_24fps.mp4", 
    output_video_path = "/home/bohbot/Evyatar/exp/Joni_showoff/Rertiles/rept.mp4"
)


Annotated video saved as: /home/bohbot/Evyatar/exp/Joni_showoff/Rertiles/rept.mp4
